#### 1. Importing regular python libraries

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk import word_tokenize,sent_tokenize
import pandas as pd
import string

#### 2. Get the Data

In [2]:
!dir data

 Volume in drive C has no label.
 Volume Serial Number is 9CDE-5548

 Directory of C:\Users\User\Desktop\CHHABI\NLP\Next_Word_Generator\data

02/06/2025  11:12 AM    <DIR>          .
02/15/2025  10:25 PM    <DIR>          ..
12/13/2023  03:33 PM           170,568 next_word_predictor.txt
               1 File(s)        170,568 bytes
               2 Dir(s)  100,501,012,480 bytes free


In [23]:
with open("data/next_word_predictor.txt","r") as f:
    data = f.readlines()

#### 2.1 Tokenize the words and Assign each unique word with index

In [25]:
main_data = " ".join(i.replace("\n"," ").strip() for i in data)

In [32]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts([main_data])
tokens_index = tokenizer.word_index
len(tokenizer.word_index)
tokens_index

{'the': 1,
 'and': 2,
 'a': 3,
 'of': 4,
 'to': 5,
 'i': 6,
 'you': 7,
 'in': 8,
 'is': 9,
 'monica': 10,
 'it': 11,
 'with': 12,
 'ross': 13,
 'that': 14,
 'rachel': 15,
 'for': 16,
 'chandler': 17,
 'this': 18,
 'on': 19,
 'joey': 20,
 'was': 21,
 'oh': 22,
 'phoebe': 23,
 'are': 24,
 'all': 25,
 'as': 26,
 'what': 27,
 'be': 28,
 'like': 29,
 'no': 30,
 "it's": 31,
 "i'm": 32,
 'her': 33,
 'they': 34,
 'just': 35,
 'from': 36,
 'okay': 37,
 'not': 38,
 'so': 39,
 'my': 40,
 'have': 41,
 'me': 42,
 'where': 43,
 'know': 44,
 'she': 45,
 'we': 46,
 'out': 47,
 'well': 48,
 'their': 49,
 'can': 50,
 'at': 51,
 'he': 52,
 'yeah': 53,
 'your': 54,
 'about': 55,
 'but': 56,
 'its': 57,
 'up': 58,
 "don't": 59,
 'text': 60,
 'scene': 61,
 'by': 62,
 'do': 63,
 'an': 64,
 'or': 65,
 'were': 66,
 'there': 67,
 'if': 68,
 'uh': 69,
 'look': 70,
 'life': 71,
 'through': 72,
 'into': 73,
 'him': 74,
 'his': 75,
 "you're": 76,
 'hey': 77,
 'how': 78,
 'right': 79,
 'think': 80,
 'time': 81,
 'no

#### 2.2 Generate the sentence tokens and convert text or sentences to sequences

In [6]:
def text_to_sequence(input_text,tokens_index):
    return [tokens_index[word] for word in input_text.split() if word in tokens_index]

input_sequence = []
for sentence in main_data.split('.'):
    sentence_token = text_to_sequence(sentence , tokens_index)   #### now replace all the words of the sentence with the token index
    for i in range(1,len(sentence_token)):  ##### 2.3 Now convert the data into input and output form like n-grams
        input_sequence.append(sentence_token[:i+1])

In [7]:
input_sequence

[[155, 21],
 [155, 21, 2368],
 [155, 21, 2368, 1549],
 [155, 21, 2368, 1549, 8],
 [155, 21, 2368, 1549, 8, 1],
 [155, 21, 2368, 1549, 8, 1, 422],
 [155, 21, 2368, 1549, 8, 1, 422, 692],
 [155, 21, 2368, 1549, 8, 1, 422, 692, 2],
 [155, 21, 2368, 1549, 8, 1, 422, 692, 2, 3],
 [155, 21, 2368, 1549, 8, 1, 422, 692, 2, 3, 2369],
 [155, 21, 2368, 1549, 8, 1, 422, 692, 2, 3, 2369, 1550],
 [155, 21, 2368, 1549, 8, 1, 422, 692, 2, 3, 2369, 1550, 2370],
 [155, 21, 2368, 1549, 8, 1, 422, 692, 2, 3, 2369, 1550, 2370, 1],
 [155, 21, 2368, 1549, 8, 1, 422, 692, 2, 3, 2369, 1550, 2370, 1, 423],
 [155, 21, 2368, 1549, 8, 1, 422, 692, 2, 3, 2369, 1550, 2370, 1, 423, 4],
 [155, 21, 2368, 1549, 8, 1, 422, 692, 2, 3, 2369, 1550, 2370, 1, 423, 4, 1],
 [155,
  21,
  2368,
  1549,
  8,
  1,
  422,
  692,
  2,
  3,
  2369,
  1550,
  2370,
  1,
  423,
  4,
  1,
  1142],
 [155,
  21,
  2368,
  1549,
  8,
  1,
  422,
  692,
  2,
  3,
  2369,
  1550,
  2370,
  1,
  423,
  4,
  1,
  1142,
  491],
 [66, 47],
 [66,

#### 2.3 In Machine Learning the input data size should be equal so we need to add padding to the each input_sequence to make the equal size

In [8]:
#### For this we need to find the maximum length of the input sequence
max_len = max([len(sequence) for sequence in input_sequence])

In [9]:
max_len

53

In [10]:
import tensorflow
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_input_sequence = pad_sequences(input_sequence, maxlen = max_len, padding = 'pre')

In [11]:
padded_input_sequence.shape

(17371, 53)

In [12]:
padded_input_sequence

array([[   0,    0,    0, ...,    0,  155,   21],
       [   0,    0,    0, ...,  155,   21, 2368],
       [   0,    0,    0, ...,   21, 2368, 1549],
       ...,
       [   0,    0,    0, ..., 2331,  290,   19],
       [   0,    0,    0, ...,  290,   19,   54],
       [   0,    0,    0, ...,   19,   54, 1535]], dtype=int32)

##### 2.3.1 Split the data into input and output

In [13]:
X = padded_input_sequence[:, : -1]

In [14]:
y = padded_input_sequence[:,-1]

##### Since our text generator model is multi class classifier we convert the y (output) to the One Hot Encoder

In [15]:
y.shape

(17371,)

In [16]:
X.shape

(17371, 52)

In [17]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes=4994)

In [18]:
y.shape

(17371, 4994)

#### 3. Now Create the Model Architecture

In [35]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [36]:
model = Sequential()
model.add(Embedding(4994, 100, input_shape=(52,)))  #### embeddings in NLP convert sparse vectors to dense vectors.
model.add(LSTM(150))
model.add(Dense(4994, activation='softmax'))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [37]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (None, 52, 100)             │         499,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 150)                 │         150,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 4994)                │         754,094 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,404,094 (5.36 MB)

 Trainable params: 1,404,094 (5.36 MB)

 Non-trainable params: 0 (0.00 B)

##### Code in pytorch for model building

In [38]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torchinfo

# class LSTMModel(nn.Module):
#     def __init__(self, vocab_size=4994, embed_dim=100, hidden_dim=150, output_dim=4994):
#         super(LSTMModel, self).__init__()
#         self.embedding = nn.Embedding(vocab_size, embed_dim)
#         self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
#         self.fc = nn.Linear(hidden_dim, output_dim)
#         self.softmax = nn.Softmax(dim=1)

#     def forward(self, x):
#         x = self.embedding(x)
#         x, _ = self.lstm(x)
#         x = x[:, -1, :]  # Take the last time step output
#         x = self.fc(x)
#         x = self.softmax(x)
#         return x

# # Define model, loss, and optimizer
# model = LSTMModel()
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters())


### 4. Train the data

In [39]:
model.fit(X,y,epochs = 30)

Epoch 1/100
543/543 ━━━━━━━━━━━━━━━━━━━━ 15s 25ms/step - accuracy: 0.0593 - loss: 7.1538
Epoch 2/100
543/543 ━━━━━━━━━━━━━━━━━━━━ 14s 25ms/step - accuracy: 0.0818 - loss: 6.3445
Epoch 3/100
543/543 ━━━━━━━━━━━━━━━━━━━━ 14s 25ms/step - accuracy: 0.0957 - loss: 6.0305
Epoch 4/100
543/543 ━━━━━━━━━━━━━━━━━━━━ 14s 25ms/step - accuracy: 0.1264 - loss: 5.6811
Epoch 5/100
543/543 ━━━━━━━━━━━━━━━━━━━━ 14s 25ms/step - accuracy: 0.1454 - loss: 5.3797
Epoch 6/100
543/543 ━━━━━━━━━━━━━━━━━━━━ 14s 25ms/step - accuracy: 0.1614 - loss: 5.0598
Epoch 7/100
543/543 ━━━━━━━━━━━━━━━━━━━━ 13s 25ms/step - accuracy: 0.1766 - loss: 4.7566
Epoch 8/100
543/543 ━━━━━━━━━━━━━━━━━━━━ 13s 24ms/step - accuracy: 0.2094 - loss: 4.4066
Epoch 9/100
543/543 ━━━━━━━━━━━━━━━━━━━━ 13s 24ms/step - accuracy: 0.2236 - loss: 4.1372
Epoch 10/100
543/543 ━━━━━━━━━━━━━━━━━━━━ 13s 24ms/step - accuracy: 0.2540 - loss: 3.8715
Epoch 11/100
543/543 ━━━━━━━━━━━━━━━━━━━━ 13s 24ms/step - accuracy: 0.2933 - loss: 3.5996
Epoch 12/100
543/54

KeyboardInterrupt: 